# 10 — Gradio Demo: Time Series Forecast (Daily Temps)

In [2]:
# Install latest stable PyCaret 3.x and common GPU-capable libs
!pip -q install --upgrade pycaret xgboost catboost lightgbm plotly gradio

import sys, sklearn, pycaret, platform
print("Python:", sys.version)
print("Platform:", platform.platform())
print("scikit-learn:", sklearn.__version__)
print("pycaret:", pycaret.__version__)

# Check GPU
try:
    import subprocess, shutil
    if shutil.which("nvidia-smi"):
        print("\nGPU detected:")
        print(subprocess.check_output(["nvidia-smi"], text=True))
    else:
        print("\nNo NVIDIA GPU detected in this runtime. (Colab: Runtime -> Change runtime type -> GPU)")
except Exception as e:
    print("GPU check error:", e)

zsh:1: command not found: pip
Python: 3.11.11 (main, Feb  5 2025, 19:11:07) [Clang 19.1.6 ]
Platform: Linux-6.6.87.2-microsoft-standard-WSL2-x86_64-with-glibc2.35
scikit-learn: 1.4.2
pycaret: 3.3.2

GPU detected:
Sun Nov  2 20:15:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.65.05              Driver Version: 580.88         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  |   00000000:01:00.0  On |                  N/A |
|  0%   31C    P8  

In [4]:
import pandas as pd
from datetime import datetime, timedelta

# Load and prepare the data
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv")
df['Date'] = pd.to_datetime(df['Date'])

# Shift dates to current time period
last_date = df['Date'].max()
today = pd.Timestamp.now().normalize()
date_shift = today - last_date
df['Date'] = df['Date'] + date_shift

# Set up the time series with daily frequency
y = df.set_index('Date')['Temp'].asfreq('D')

from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()
exp.setup(
    data=y, 
    fh=14, 
    fold=3, 
    session_id=42, 
    seasonal_period=7, 
    verbose=False,
    numeric_imputation_target='mean'
)
model = exp.finalize_model(exp.compare_models())

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
croston,Croston,0.7852,0.8021,2.3174,3.0064,0.1695,0.1722,-0.1385,0.0200
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,0.7948,0.7795,2.3458,2.9218,0.1692,0.1764,-0.1296,0.1433
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.7959,0.7647,2.3488,2.8662,0.1825,0.1724,-0.0194,92.9933
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.7978,0.7821,2.3547,2.9313,0.1698,0.1772,-0.1412,0.1800
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.7978,0.7821,2.3547,2.9313,0.1698,0.1772,-0.1412,0.1467
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.7981,0.7822,2.3554,2.9320,0.1698,0.1773,-0.1415,0.1533
arima,ARIMA,0.8141,0.8101,2.4023,3.0362,0.1815,0.1833,-0.2882,0.4167
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.8182,0.7963,2.4147,2.9846,0.1723,0.1817,-0.1750,0.1467
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.8342,0.8080,2.4619,3.0285,0.1743,0.1854,-0.2039,0.1400
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.8342,0.8080,2.4619,3.0285,0.1743,0.1854,-0.2039,0.1433


In [9]:
import gradio as gr
import pandas as pd
import numpy as np
import socket
from datetime import datetime, timedelta

def find_free_port(start_port=7860, max_port=7960):
    """Find a free port in the given range"""
    for port in range(start_port, max_port):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('127.0.0.1', port))
                return port
            except OSError:
                continue
    raise OSError(f"No free ports found in range {start_port}-{max_port}")

def _extract_forecast(preds):
    """Return (index, 1d-array) from various predict_model return types.

    Handles Series, DataFrame (with 'y_pred' or first column), PeriodIndex, DatetimeIndex,
    and numpy arrays.
    """
    # If predict_model returns a pandas Series
    if isinstance(preds, pd.Series):
        idx = preds.index
        vals = preds.values
        # Convert PeriodIndex to timestamp index
        if isinstance(idx, pd.PeriodIndex):
            idx = idx.to_timestamp()
        return idx, np.asarray(vals).reshape(-1)

    # If it's a DataFrame with a y_pred column (PyCaret typical)
    if isinstance(preds, pd.DataFrame):
        # If it has a PeriodIndex, convert
        idx = preds.index
        if isinstance(idx, pd.PeriodIndex):
            idx = idx.to_timestamp()
        if 'y_pred' in preds.columns:
            vals = preds['y_pred'].values
            return idx, np.asarray(vals).reshape(-1)
        # fallback: take first column
        vals = preds.iloc[:, 0].values
        return idx, np.asarray(vals).reshape(-1)

    # Last resort: try to convert to numpy (e.g., array-like)
    try:
        arr = np.asarray(preds)
        if arr.ndim == 1:
            # create synthetic index starting after last training date
            last_date = y.index.max()
            freq = y.index.freq if getattr(y.index, 'freq', None) is not None else pd.tseries.frequencies.to_offset('D')
            # ensure freq is a DateOffset
            if not isinstance(freq, pd.DateOffset):
                freq = pd.tseries.frequencies.to_offset(freq)
            idx = pd.date_range(start=last_date + freq, periods=arr.shape[0], freq=freq)
            return idx, arr.reshape(-1)
    except Exception:
        pass
    raise ValueError('Unable to extract forecast values from model output')


def forecast(h=14):
    try:
        # Ensure h is a valid integer (PyCaret expects int or list of ints)
        if isinstance(h, (list, tuple)):
            # if user somehow passes list, use its length as horizon
            fh = int(len(h))
        else:
            fh = int(h)
        if fh < 1:
            return pd.DataFrame({"Error": ["Forecast horizon must be positive"]})

        # Use integer horizon for predict_model (required type)
        preds = exp.predict_model(model, fh=fh)

        # Extract index and 1D forecast values robustly
        idx, values = _extract_forecast(preds)

        # Normalize index to DatetimeIndex if PeriodIndex slipped through
        if isinstance(idx, pd.PeriodIndex):
            idx = idx.to_timestamp()
        idx = pd.to_datetime(idx)

        # Build output dataframe combining last 7 historical + forecast
        # Historical (last 7 days)
        historical = y.tail(7)
        hist_df = pd.DataFrame({
            "Date": historical.index.strftime('%Y-%m-%d'),
            "Forecast Temperature": historical.values.round(2),
            "Type": "Historical"
        })

        # Forecast
        forecast_df = pd.DataFrame({
            "Date": idx.strftime('%Y-%m-%d'),
            "Forecast Temperature": values.round(2),
            "Type": "Forecast"
        })

        combined_df = pd.concat([hist_df, forecast_df], ignore_index=True)
        return combined_df
    except Exception as e:
        # Return error to UI and print for debugging
        print(f"Error in forecast: {str(e)}")
        return pd.DataFrame({"Error": [str(e)]})

# Create the interface with improved styling
demo = gr.Interface(
    fn=forecast,
    inputs=gr.Slider(
        minimum=7,
        maximum=60,
        value=14,
        step=1,
        label="Forecast Horizon (days)"
    ),
    outputs=gr.Dataframe(label="Temperature Forecast"),
    title="Daily Temperature Forecaster",
    description="Shows last 7 days of historical data and forecasted temperatures for the selected number of days.",
    theme="default"
)

# Find an available port and launch
try:
    port = find_free_port()
    print(f"Starting server on port {port}")
    demo.launch(
        server_name="127.0.0.1",
        server_port=port,
        share=False,
        show_error=True
    )
except Exception as e:
    print(f"Error launching server: {str(e)}")

Starting server on port 7866
* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.
